In [2]:
import cv2 
import numpy as np
from IPython.display import display, Markdown

In [3]:
net = cv2.dnn.readNet("yolov3-tiny.weights","yolov3-tiny.cfg")

In [4]:
# Classes 
classe = []

with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
display(Markdown("### Classes do coco"))
display(classes)

layer_names = net.getLayerNames()

outputLayers = [layer_names[i- 1] for i in net.getUnconnectedOutLayers()]
display(Markdown("### Output Layers"))
display(outputLayers)

### Classes do coco

['person',
 'bicycle',
 'car',
 'motorcycle',
 'airplane',
 'bus',
 'train',
 'truck',
 'boat',
 'traffic light',
 'fire hydrant',
 'stop sign',
 'parking meter',
 'bench',
 'bird',
 'cat',
 'dog',
 'horse',
 'sheep',
 'cow',
 'elephant',
 'bear',
 'zebra',
 'giraffe',
 'backpack',
 'umbrella',
 'handbag',
 'tie',
 'suitcase',
 'frisbee',
 'skis',
 'snowboard',
 'sports ball',
 'kite',
 'baseball bat',
 'baseball glove',
 'skateboard',
 'surfboard',
 'tennis racket',
 'bottle',
 'wine glass',
 'cup',
 'fork',
 'knife',
 'spoon',
 'bowl',
 'banana',
 'apple',
 'sandwich',
 'orange',
 'broccoli',
 'carrot',
 'hot dog',
 'pizza',
 'donut',
 'cake',
 'chair',
 'couch',
 'potted plant',
 'bed',
 'dining table',
 'toilet',
 'tv',
 'laptop',
 'mouse',
 'remote',
 'keyboard',
 'cell phone',
 'microwave',
 'oven',
 'toaster',
 'sink',
 'refrigerator',
 'book',
 'clock',
 'vase',
 'scissors',
 'teddy bear',
 'hair drier',
 'toothbrush']

### Output Layers

['yolo_16', 'yolo_23']

In [5]:
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [7]:
cap = cv2.VideoCapture('1.mp4')

while True:
    ret, frame = cap.read()
    height, width, channels = frame.shape

    blob = cv2.dnn.blobFromImage(frame, 0.00392, (320, 320), (0,0,0), True)
    net.setInput(blob)
    outs = net.forward(outputLayers)

    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0]*width)
                center_y = int(detection[1]*height)
                w = int(detection[2]*width)
                h = int(detection[3]*height)

                # Coordenadas do retângulo
                x = int(center_x - w/2)
                y = int(center_y - h/2)

                boxes.append([x,y,w,h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.4, 0.6)


    for i in range(len(boxes)):
        if i in indexes:
            x,y,w,h = boxes[i]
            label = str(classes[class_ids[i]])
            color = colors[class_ids[i]]
            cv2.rectangle(frame, (x,y), (x+w, y+h), color, 2)
            cv2.putText(frame, label+" " + str(round(confidence, 2)), (x, y+30), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,255),2)
            cv2.imshow("Image", frame)
    if cv2.waitKey(1) == 27:
            break
cap.release()
cv2.destroyAllWindows()